In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
import pandas as pd

In [ ]:
# Read data from Excel file into a Pandas DataFrame
file_path = 'dm_mimic_pathways.csv'
df = pd.read_csv(file_path)

In [ ]:
column_name_mapping = {'person_id': 'Person',
                       'race_concept_id': 'Race',
                       'gender_concept_id':'Gender',
                       'age_group':'Age Group',
                       'pathways':'Treatment Regimen'}

race_mapping = {8527: 'White/Hispanic',
                8516: 'Black',
                8515: 'Asian', 
                0:'Unknown',
                38003592: 'Asian',
                4077359: 'Other',
                4218674: 'Unknown',
                4188159: 'White/Hispanic',
                38003599: 'Black',
                38003574: 'Asian',
                4212311: 'Asian',
                38003600: 'Black',
                8557: 'Other',
                38003584: 'Asian',
                38003578: 'Asian',
                4087921: 'Other',
                38003615: 'Other',
                38003581: 'Asian',
                8657: 'Other',
                38003579: 'Asian',
                38003605: 'Black',
                38003614: 'White/Hispanic',
                4213463: 'White/Hispanic'}

gender_mapping = {8507: 'Male',
                  8532: 'Female'}

age_mapping = {'10 - 19': 'Teens',
               '20 - 29': 'Twenties',
               '30 - 39': 'Thirties',
               '40 - 49': 'Forties',
               '50 - 59': 'Fifties',
               '60 - 69': 'Sixties',
               '70 - 79': 'Seventies',
               '80 - 89': 'Eighties',
              '> 90': 'Nineties'}

In [ ]:
df = df.rename(columns=column_name_mapping)
df['Race'] = df['Race'].replace(race_mapping)
df['Gender'] = df['Gender'].replace(gender_mapping)
df['Age Group'] = df['Age Group'].replace(age_mapping)
df['Age Group'].fillna('Unknown', inplace=True)

In [ ]:
df = df[(df['Age Group'] != 'Unknown') & (df['Race'] != 'Unknown') & (df['Treatment Regimen'] != 'Undefined')]
df.head(3)

In [ ]:
predictors = ['Gender', 'Age Group', 'Race']
response = 'Treatment Regimen'

In [ ]:
print(len(df))
n = 2
values_to_preserve = df[response].value_counts().head(n)
print(values_to_preserve)

In [ ]:
def preserve_or_change(value, value_set, replacement_value):
    return value if value in value_set else replacement_value

In [ ]:
df[response] = df[response].apply(lambda x: preserve_or_change(x, values_to_preserve, 'Other'))
df.head(5)
len(df[response].unique())

In [ ]:
X = df[predictors]
y = df[response]

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
            ('cat', OneHotEncoder(drop = 'first'), predictors)
        ],
        remainder='passthrough'
    
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(multi_class='multinomial', class_weight = 'balanced'))
])

In [ ]:
# Initialize LeaveOneOut cross-validator
loo = LeaveOneOut()

# Lists to store predicted and true labels
predicted_labels = []
true_labels = []

# Perform LOOCV
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model on the training set
    pipeline.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = pipeline.predict(X_test)

    # Store predicted and true labels
    predicted_labels.extend(y_pred)
    true_labels.extend(y_test)

# Evaluate accuracy on the entire dataset
accuracy_loocv = accuracy_score(true_labels, predicted_labels)


print("Accuracy using LOOCV:", accuracy_loocv)

In [ ]:
# # Specify the number of folds (e.g., 5-fold cross-validation)
# num_folds = 5

# # Perform cross-validation
# cv_scores = cross_val_score(pipeline, X, y, cv=num_folds, scoring=make_scorer(accuracy_score))

# # Print cross-validation scores
# print("Cross-validation scores:", cv_scores)
# print("Mean accuracy:", cv_scores.mean())

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train the model
pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Create a DataFrame with actual and predicted values
df_predictions = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print("Actual vs Predicted:")
print(df_predictions)

In [ ]:
# Access the one-hot encoder from the pipeline
encoder = pipeline.named_steps['preprocessor'].named_transformers_['cat']

# Get feature names after one-hot encoding
feature_names_after_encoding = list(encoder.get_feature_names_out(X.select_dtypes(include=['object']).columns))

# Concatenate feature names with numeric features
all_feature_names = X.select_dtypes(include=['number']).columns.tolist() + feature_names_after_encoding

# Access the model from the pipeline
model = pipeline.named_steps['classifier']

# Get coefficients
coefficients = model.coef_

# Display coefficients in a DataFrame
df_coefficients = pd.DataFrame(coefficients, columns=all_feature_names)
df_coefficients['Intercept'] = model.intercept_
df_coefficients['Class'] = model.classes_
df_coefficients.set_index('Class', inplace=True)

print("Coefficients:")
print(df_coefficients)